In [16]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk

In [17]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

Dataset_short=Dataset[:,0:15,:]

#for i, batch in enumerate(Dataset):
#  pt_sum=0
#  for j, particle in enumerate(Dataset[i,:,:]):
#    if particle[3]!=0:
#      pt_sum+=particle[0]
#  for j, particle in enumerate(Dataset[i,:,:]):
#    particle[0]=particle[0]/pt_sum
    
    
Data_Train=Dataset_short[0:2000000,:,0:3]
Data_Test=Dataset_short[2000001:3600000,:,0:3]
Data_Validate=Dataset_short[3600001:4000000,:,0:3]



Data_Train_Flat=np.reshape(Data_Train,(-1,45))
Data_Val_Flat=np.reshape(Data_Validate,(-1,45))

In [24]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_encoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    x=layers.Dense(h_dim_2, activation='relu')(x)
    z_mean=layers.Dense(latent_dim, name='z_mean')(x)
    z_logvar=layers.Dense(latent_dim, name='z_log_var')(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder




def make_decoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu')(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    z=layers.Dense(input_dim)(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

class VAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
    

In [25]:
#Here is a normalized model with (1-beta)rl beta*Kl loss
beta=0.83
vae_enc=make_encoder(45,32,16,3)
vae_dec=make_decoder(45,32,16,3)
vae_40MHZ=VAE_Model(vae_enc,vae_dec)
vae_40MHZ.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
vae_40MHZ.compile(optimizer=opt)


STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = vae_40MHZ.fit(x=Data_Train_Flat, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True)
vae_40MHZ.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/Synthesis_testing/reduced_size_model/'.format(beta),save_format='tf')

Epoch 1/100
18000/18000 [==============================] - 22s 1ms/step - loss: 11159987.0154 - reco_loss: 2233984.3178 - kl_loss: 1255665.6250 - val_loss: 1240.7700 - val_reco_loss: 14.4556 - val_kl_loss: 1226.3145 - lr: 0.0010
Epoch 2/100
18000/18000 [==============================] - 19s 1ms/step - loss: 26.2663 - reco_loss: 22.5746 - kl_loss: 3.4165 - val_loss: 872.2685 - val_reco_loss: 11.2195 - val_kl_loss: 861.0490 - lr: 0.0010
Epoch 3/100
18000/18000 [==============================] - 19s 1ms/step - loss: 20.9972 - reco_loss: 17.9928 - kl_loss: 2.9637 - val_loss: 833.8842 - val_reco_loss: 9.4429 - val_kl_loss: 824.4412 - lr: 0.0010
Epoch 4/100
18000/18000 [==============================] - 19s 1ms/step - loss: 20.1711 - reco_loss: 17.1166 - kl_loss: 2.9180 - val_loss: 781.0735 - val_reco_loss: 21.8116 - val_kl_loss: 759.2620 - lr: 0.0010
Epoch 5/100
18000/18000 [==============================] - 19s 1ms/step - loss: 18.1801 - reco_loss: 15.4191 - kl_loss: 2.7461 - val_loss: 778

In [26]:
a=vae_40MHZ.predict(np.zeros((1,45)))
vae_40MHZ.encoder.summary()
vae_40MHZ.decoder.summary()

1/1 [==============================] - 0s 122ms/step
Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 45)]                 0         []                            
                                                                                                  
 batch_normalization_4 (Bat  (None, 45)                   180       ['input_11[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 dense_25 (Dense)            (None, 32)                   1472      ['batch_normalization_4[0][0]'
                                                                    ]                             
                                       